# Analysis

There are two classes for performing an analysis: Cost Benefit Analysis and Multiple Criteria Analysis. 

## Cost Benefit Analysis

In [ ]:
from dpd.analysis import CostBenefitAnalysis

In [2]:
discount_rate = 0.03
start_year = 2017
duration = 3  # years

In [3]:
cba = CostBenefitAnalysis(start_year=start_year, duration=duration)
cba.add_cost(name="cost 1", value=10, start_year=start_year, duration=2)
cba.add_cost(name="cost 2", value=10, start_year=start_year, duration=2)
cba.add_benefit(name="benefit 1", value=20, start_year=start_year + 1, duration=2)
cba.add_benefit(name="benefit 2", value=20, start_year=start_year + 1, duration=2)

# cba_table = cba.to_dataframe().T # in year of expenditure
cba_table = cba.discount(start_year, discount_rate).T
print(
    "Benefit-cost ratio: ",
    cba_table["Sum"]["Benefits Total"] / cba_table["Sum"]["Costs Total"],
)
cba_table

Benefit-cost ratio:  1.9417475728155342


,2017,2018,2019,Sum
cost 1,10.0,9.708738,0.000000,19.708738
cost 2,10.0,9.708738,0.000000,19.708738
Costs Total,20.0,19.417476,0.000000,39.417476
benefit 1,0.0,19.417476,18.851918,38.269394
benefit 2,0.0,19.417476,18.851918,38.269394
Benefits Total,0.0,38.834951,37.703836,76.538788
Benefits - Costs Total,-20.0,19.417476,37.703836,37.121312


## Multiple-criteria Analysis

In [1]:
from dpd.analysis import MultipleCriteriaAnalysis
from dpd.d3 import radar_chart
from IPython.core.display import HTML

In [6]:
attributes = ["Cost", "Time", "Comfort"]
alternatives = ["Tram", "Bus"]

mca = MultipleCriteriaAnalysis(attributes, alternatives)
mca.mca["Tram"]["Cost"] = 200
mca.mca["Bus"]["Cost"] = 100
mca.mca["Tram"]["Time"] = 50
mca.mca["Bus"]["Time"] = 100
mca.mca["Tram"]["Comfort"] = 800
mca.mca["Bus"]["Comfort"] = 500

mca.mca

,Tram,Bus
Cost,200.0,100.0
Time,50.0,100.0
Comfort,800.0,500.0


In [7]:
legend_options, d, title = mca.to_d3_radar_chart()
HTML(radar_chart(legend_options, d, title))